In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os

# Set GPU to 0
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from MIMIC_IV_HAIM_API import split_note_document, get_biobert_embeddings

2023-12-05 11:49:24.218259: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 11:49:24.218285: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 11:49:24.218316: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 11:49:24.224973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/cis/home/charr165/vscode_projects/HAIM/.venv/lib/

In [3]:
mm_dir = "/cis/home/charr165/Documents/multimodal"
output_dir = os.path.join(mm_dir, "preprocessing")

rad_notes_df = pd.read_pickle(os.path.join(output_dir, "notes_text.pkl"))

In [4]:
icu_rad_notes_df = rad_notes_df[rad_notes_df['stay_id'].notna()]

In [5]:
from tqdm import tqdm

icu_rad_notes_df['biobert_embeddings'] = None
for index, row in tqdm(icu_rad_notes_df.iterrows(), total=icu_rad_notes_df.shape[0]):
    curr_subject_id = int(row['subject_id'])
    curr_note_id =row['note_id']
    curr_text = row['text']

    chunk_parse, chunk_length = split_note_document(curr_text, 15)

    embeddings = []
    for chunk in chunk_parse:
        curr_embeddings, _ = get_biobert_embeddings(chunk)
        embeddings.append(curr_embeddings)
    
    icu_rad_notes_df.at[index, 'biobert_embeddings'] = embeddings



100%|██████████| 282788/282788 [4:16:08<00:00, 18.40it/s]  


In [6]:
icu_rad_notes_df.to_pickle(os.path.join(output_dir, "icu_notes_text_embeddings.pkl"))